In [33]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [34]:
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark


In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls


number.txt		   spark-2.3.1-bin-hadoop2.7.tgz    textfile.txt
sample_data		   spark-2.3.1-bin-hadoop2.7.tgz.1
spark-2.3.1-bin-hadoop2.7  spark-warehouse


In [36]:
import findspark
findspark.init()


In [37]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("application").getOrCreate() 
spark

In [38]:
df = spark.createDataFrame([{"spark on": "colab"} for x in range(1000)])
df.show(1)

/content/spark-2.3.1-bin-hadoop2.7/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+
|spark on|
+--------+
|   colab|
+--------+
only showing top 1 row



In [39]:
#1.	Create RDDs in three different ways.

In [40]:
#1st method
text="hello. wellcome to manipal school of information science. there are so many branch are there, like ML, BDA, CLOUD, VLSI, HBA.".split(" ")
words = spark.sparkContext.parallelize(text,2)
type(words)
words.collect()

['hello.',
 'wellcome',
 'to',
 'manipal',
 'school',
 'of',
 'information',
 'science.',
 'there',
 'are',
 'so',
 'many',
 'branch',
 'are',
 'there,',
 'like',
 'ML,',
 'BDA,',
 'CLOUD,',
 'VLSI,',
 'HBA.']

In [41]:
#2nd method
rdd_2=spark.sparkContext.textFile("textfile.txt")
rdd_2.collect()
type(rdd_2)

pyspark.rdd.RDD

In [42]:
#3rd method
rdd_3=words.filter(lambda word:word.startswith('s'))
rdd_3.collect()
type(rdd_3)

pyspark.rdd.PipelinedRDD

In [43]:
#2.Read a text file and count the number of words in the file using RDD operations.

In [44]:
word_count=rdd_2.flatMap(lambda s:s.split(" "))
word_count.count()

21

In [45]:
#3.Write a program to find the word frequency in a given file.

In [46]:
info=spark.sparkContext.textFile("textfile.txt")
info=info.flatMap(lambda x:x.split())
content_map=info.map(lambda c:(c,1))
content_map.reduceByKey(lambda a,b:a+b).collect()

[('school', 1),
 ('of', 1),
 ('science.', 1),
 ('there', 1),
 ('are', 2),
 ("branch's", 1),
 ('there,', 1),
 ('like', 1),
 ('ML,', 1),
 ('BDA,', 1),
 ('CLOUD,', 1),
 ('HBA.', 1),
 ('hello.', 1),
 ('wellcome', 1),
 ('to', 1),
 ('manipal', 1),
 ('information', 1),
 ('so', 1),
 ('many', 1),
 ('VLSI,', 1)]

In [47]:
#4.Write a program to convert all words in a file to uppercase.

In [48]:
rdd_2.map(lambda c:c.upper()).collect()

['HELLO. WELLCOME TO MANIPAL SCHOOL OF INFORMATION SCIENCE.',
 "THERE ARE SO MANY BRANCH'S ARE THERE, LIKE ML, BDA, CLOUD, VLSI, HBA."]

5.Write a program to convert all words in a file to lowercase.


In [49]:
#5.Write a program to convert all words in a file to lowercase.

In [50]:
rdd_2.map(lambda c:c.lower()).collect()

['hello. wellcome to manipal school of information science.',
 "there are so many branch's are there, like ml, bda, cloud, vlsi, hba."]

6.Write a program to capitalize first letter of each words in file (use string capitalize() method).


In [51]:
capital=rdd_2.flatMap(lambda a:a.split(" ")).map(lambda c:c.capitalize()).collect()
" ".join(capital)

"Hello. Wellcome To Manipal School Of Information Science. There Are So Many Branch's Are There, Like Ml, Bda, Cloud, Vlsi, Hba."

7.Find the longest length of word from given set of words.


In [52]:
longest_word=rdd_2.flatMap(lambda x:x.split(" "))
longest_word.map(lambda nu:(len(nu),nu)).max()[1]

'information'

8.Map the Registration numbers to corresponding branch. 6000 series BDA, 9000 series HAD, 1000 series MS, 2000 series VLSI, 3000 series ES, 4000 series MSc, 5000 series CC. Given registration number, generate a key-value pair of Registration Number and Corresponding Branch.

In [53]:
registration_number=[6027,2005,2035,6011,9007,9056,3088,3045,4088,4065,5077,5066,1001,1002]
context=spark.sparkContext.parallelize(registration_number,2)
classify=context.map(lambda reg:('VLSI',reg) if reg>2000 and reg<3000 
        else ('MS',reg) if reg>1000 and reg<2000
        else ('ES',reg) if reg>3000 and reg<4000
        else ('MSc',reg) if reg>4000 and reg<5000
        else ('CC',reg) if reg>5000 and reg<6000
        else ('BDA',reg) if reg>6000 and reg<7000
        else ('HDA',reg))
classify.collect()

[('BDA', 6027),
 ('VLSI', 2005),
 ('VLSI', 2035),
 ('BDA', 6011),
 ('HDA', 9007),
 ('HDA', 9056),
 ('ES', 3088),
 ('ES', 3045),
 ('MSc', 4088),
 ('MSc', 4065),
 ('CC', 5077),
 ('CC', 5066),
 ('MS', 1001),
 ('MS', 1002)]

9.Text file contain numbers. Numbers are separated by one white space. 
There is no order to store the numbers. One line may contain one or more numbers.
Find the maximum, minimum, sum and mean of numbers.


In [56]:
file1=spark.sparkContext.textFile("number.txt")
file1_rdd=file1.flatMap(lambda z:z.split(" ")).map(lambda c:int(c))
file1_rdd.max()

217

In [57]:
file1_rdd.min()

8

In [ ]:
file1_rdd.sum()

In [59]:
file1_rdd.mean()

64.5

10. A text file (citizen.txt) contains data about citizens of country. Fields (information in file) are Name, dob, Phone, email and state name. Another file contains mapping of state names to state code like Karnataka is codes as KA, TamilNadu as TN, Kerala KL etc. Compress the citizen.txt file by changing full state name to state code.

In [62]:
details=spark.sparkContext.textFile("citizen.txt")
code=spark.sparkContext.textFile("state_codes_of_different_states.txt")
details_rdd=details.map(lambda x:x.split(",")).collect()
code_rdd=code.map(lambda y:y.split(",")).collect()
for i in range(len(details_rdd)):
    for j in range(len(code_rdd)):  
        if details_rdd[i][4]==code_rdd[j][0]:
            details_rdd[i][4]=code_rdd[j][1]
details_rdd              

[['thejas', '12-05-1996', '86573', 'thejas96@gmail.com', 'KA'],
 ['yashwanth', '01-02-1998', '98786', 'yashwanth@gmail.com', 'JK'],
 ['Sharanya', '31-01-1996', '98675', 'sharanya@gmail.com', 'AP'],
 ['Chethana', '12-03-1995', '89523', 'chethana@gmail.com', 'TN'],
 ['Akshith', '04-07-1993', '78234', 'akshith@gmail.com', 'GJ'],
 ['Amruta', '21-10-1996', '65432', 'amruta@gmail.com', 'WB'],
 ['Anupam', '05-09-1993', '87474', 'anupam@gmail.com', 'RJ'],
 ['Chandan', '13-02-1997', '76893', 'chandan@gmail.com', 'PB'],
 ['Himana', '04-08-1995', '98234', 'himana@gmail.com', 'JK'],
 ['Varsha', '05-09-1997', '86574', 'varsha@gmail.com', 'MH'],
 ['Arun', '17-12-1994', '67895', 'arun@gmail.com', 'UP']]

In [63]:
stRDD = spark.sparkContext.textFile('state_codes_of_different_states.txt')
stateKey = stRDD.map(lambda word: (word.split(',')[0], word.split(',')[1]))

In [64]:
#creating dictionary
code_dict = {}
for val in stateKey.collect():
    code_dict[val[0]] = val[1]
    
code_dict

{'AndhraPradesh': 'AP',
 'Gujarat': 'GJ',
 'JammuKashmir': 'JK',
 'Karnataka': 'KA',
 'Maharashtra': 'MH',
 'Punjab': 'PB',
 'Rajasthan': 'RJ',
 'TamilNadu': 'TN',
 'UttarPradesh': 'UP',
 'WestBengal': 'WB'}

In [66]:
mapData = spark.sparkContext.broadcast(code_dict)

cityRdd = spark.sparkContext.textFile('citizen.txt')
print(cityRdd.collect())
def abc(state,codes):
    splitData = state.split(',')  
    print(splitData)
    splitData[4] = codes.value.get(splitData[4])
    newData = ' '
    newData = newData.join(splitData)
    
    return newData
    
mapCitizen = cityRdd.map(lambda data: abc(data,mapData))
mapCitizen.collect()

['thejas,12-05-1996,86573,thejas96@gmail.com,Karnataka', 'yashwanth,01-02-1998,98786,yashwanth@gmail.com,JammuKashmir', 'Sharanya,31-01-1996,98675,sharanya@gmail.com,AndhraPradesh', 'Chethana,12-03-1995,89523,chethana@gmail.com,TamilNadu', 'Akshith,04-07-1993,78234,akshith@gmail.com,Gujarat', 'Amruta,21-10-1996,65432,amruta@gmail.com,WestBengal', 'Anupam,05-09-1993,87474,anupam@gmail.com,Rajasthan', 'Chandan,13-02-1997,76893,chandan@gmail.com,Punjab', 'Himana,04-08-1995,98234,himana@gmail.com,JammuKashmir', 'Varsha,05-09-1997,86574,varsha@gmail.com,Maharashtra', 'Arun,17-12-1994,67895,arun@gmail.com,UttarPradesh']


['thejas 12-05-1996 86573 thejas96@gmail.com KA',
 'yashwanth 01-02-1998 98786 yashwanth@gmail.com JK',
 'Sharanya 31-01-1996 98675 sharanya@gmail.com AP',
 'Chethana 12-03-1995 89523 chethana@gmail.com TN',
 'Akshith 04-07-1993 78234 akshith@gmail.com GJ',
 'Amruta 21-10-1996 65432 amruta@gmail.com WB',
 'Anupam 05-09-1993 87474 anupam@gmail.com RJ',
 'Chandan 13-02-1997 76893 chandan@gmail.com PB',
 'Himana 04-08-1995 98234 himana@gmail.com JK',
 'Varsha 05-09-1997 86574 varsha@gmail.com MH',
 'Arun 17-12-1994 67895 arun@gmail.com UP']